### Import libraries

#### Run the same model and add the correctly labeled tweets based on the prediction on the fifth model
- I used KNN to predict the fifth model. Out of the 25 test data, 5 were correct ~ 20%.
- I took 25 test tweets  in the previous model because I want to focus on the negative sentiments to reflect 0 value.
- 0: 198
- 1: 396

In [1]:
import numpy as np
import pandas as pd
import scipy as sp   
from sklearn import metrics
from textblob import TextBlob, Word
from nltk.stem.snowball import SnowballStemmer
%matplotlib inline

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.cross_validation import train_test_split #how does this work?
from sklearn.model_selection import train_test_split as tts

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier , VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
neighbors = 5
from sklearn.naive_bayes import MultinomialNB
from nltk.classify import NaiveBayesClassifier  #how does this work?

from sklearn.model_selection import cross_val_score as cvs

/Users/jedvillareal/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
kanye2 = pd.read_csv('kanyewest5.csv')
kanye2.head()

,text,label
0,RT @WSHHRAP: Rappers with the most Billboard H...,1.0
1,"Kanye and Cudi merge, together they rule as Kanye",1.0
2,RT @RappersSaid: Rappers with the most Billboa...,1.0
3,RT @c_drew_: Kanye shaped like he played barit...,1.0
4,RT @Bubbabue: @kanyewest Your album #ye is an ...,1.0


In [3]:
kanye2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2064 entries, 0 to 2063
Data columns (total 2 columns):
text     2064 non-null object
label    594 non-null float64
dtypes: float64(1), object(1)
memory usage: 32.3+ KB


In [4]:
df = kanye2.sort_values(by = 'label', ascending=True)
df1 = df.reset_index()
df1[588:].head()

,index,text,label
588,240,Haven’t really listened to Kanye’s album until...,1.0
589,239,RT @Complex: Drake receives co-writing credit ...,1.0
590,238,RT @Mitch98Mm: @mastaace Wrd! #Nobody Sad day...,1.0
591,237,RT @Complex: Drake receives co-writing credit ...,1.0
592,385,Ok but can @kanyewest acknowledge all this lov...,1.0


In [5]:
kanye2train = df1[(df1['label'] ==0)|(df1['label']==1)]
kanye2train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 594 entries, 0 to 593
Data columns (total 3 columns):
index    594 non-null int64
text     594 non-null object
label    594 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 18.6+ KB


In [6]:
kanye2train.head()

,index,text,label
0,192,@thecjpearson @kanyewest @RealCandaceO Does de...,0.0
1,130,RT @brininna: your wcw slanders Kanye for his ...,0.0
2,131,"RT @RealCandaceO: The mob is loud, but the maj...",0.0
3,132,RT @taydrug: Taylor was dragged for saying she...,0.0
4,133,"RT @RealCandaceO: The mob is loud, but the maj...",0.0


In [7]:
kanye2test = df1[df1['label'].isnull()][75:100]
kanye2test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25 entries, 669 to 693
Data columns (total 3 columns):
index    25 non-null int64
text     25 non-null object
label    0 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 800.0+ bytes


In [8]:
kanye2test.head()

,index,text,label
669,669,Cuz he’s an anti black dick head. Next. https:...,NaN
670,670,Demeter is rarely listening to Kanye West or l...,NaN
671,671,RT @BET: Is Kanye West canceled? Social media ...,NaN
672,672,RT @stankymeat: kanye: aight cudi u ready to r...,NaN
673,673,@oliverNOtwist JPrince did that to save your G...,NaN


In [9]:
text = kanye2train['text']
print(type(text))
text.head()

<class 'pandas.core.series.Series'>


0    @thecjpearson @kanyewest @RealCandaceO Does de...
1    RT @brininna: your wcw slanders Kanye for his ...
2    RT @RealCandaceO: The mob is loud, but the maj...
3    RT @taydrug: Taylor was dragged for saying she...
4    RT @RealCandaceO: The mob is loud, but the maj...
Name: text, dtype: object

In [10]:
toadd = kanye2test['text']
toadd

669    Cuz he’s an anti black dick head. Next. https:...
670    Demeter is rarely listening to Kanye West or l...
671    RT @BET: Is Kanye West canceled? Social media ...
672    RT @stankymeat: kanye: aight cudi u ready to r...
673    @oliverNOtwist JPrince did that to save your G...
674    “Are you voting for Trump/Kanye in the 2020 pr...
675    RT @TheFashionLaw: 21 rape allegations later, ...
676    RT @FoxNews: Kanye West's 'Ye' hits No. 1 on B...
677    @ibarguen @Joi @dkroy @knightfdn @BillClinton ...
678    RT @Masuta1019: And them hoes call... https://...
679    @OzLady41563 @VancityReynolds There’s a differ...
680        @jweibel24 Back when I thought Kanye was Kool
681    @WashTimes @realDonaldTrump - Please pick Kany...
682    RT @iamferd97: Man Chance dropped Acid Rap, Ky...
683    RT @balmainbratz: see how Kanye’s leg resemble...
684    Jay trying to bring the old Kanye back https:/...
685    @karenhunter @kanyewest @HISTORY Let's let Kan...
686    Did Kanye West Deserve t

### Let's try TF-IDF

In [11]:
tfidf = text.append(toadd, ignore_index = True)
tfidf.head()

0    @thecjpearson @kanyewest @RealCandaceO Does de...
1    RT @brininna: your wcw slanders Kanye for his ...
2    RT @RealCandaceO: The mob is loud, but the maj...
3    RT @taydrug: Taylor was dragged for saying she...
4    RT @RealCandaceO: The mob is loud, but the maj...
Name: text, dtype: object

In [12]:
vect = TfidfVectorizer(stop_words='english')

In [13]:
tf = pd.DataFrame(vect.fit_transform(tfidf).toarray(), columns=vect.get_feature_names())

In [14]:
tf.head()

,0jujild7px,10,100,1063atl,10s,11,12,135,148lqlf5ao,15,...,yodeling,youtube,yp6zcirlxq,ywltohwavw,yxotobksfb,zmh9qhh6xk,znu3q5mi58,zrhedjtc5k,zth5gym7xn,zuqwdrzfdd
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
tf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 619 entries, 0 to 618
Columns: 1420 entries, 0jujild7px to zuqwdrzfdd
dtypes: float64(1420)
memory usage: 6.7 MB


In [16]:
reset = kanye2train['label'].reset_index()
del reset ['index']
reset.head()

,label
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


In [17]:
kanye = pd.concat([tf,reset], axis = 1,ignore_index= False)
kanye['label'][593:].head()

593    1.0
594    NaN
595    NaN
596    NaN
597    NaN
Name: label, dtype: float64

In [18]:
edittrain = kanye[:594]
edittest = kanye[594:]
droplabel = edittest.drop('label', axis = 1)

In [19]:
edittrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 594 entries, 0 to 593
Columns: 1421 entries, 0jujild7px to label
dtypes: float64(1421)
memory usage: 6.4 MB


In [20]:
edittest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 594 to 618
Columns: 1421 entries, 0jujild7px to label
dtypes: float64(1421)
memory usage: 277.6 KB


In [21]:
x = edittrain.drop('label', axis = 1)
y = edittrain['label']

In [22]:
xtrain, xtest, ytrain, ytest = tts(x,y, random_state = 0)

In [23]:
l = LogisticRegression()
d = DecisionTreeClassifier()
r = RandomForestClassifier()
g = GradientBoostingClassifier()
k = KNeighborsClassifier()
nb = MultinomialNB()
vc = VotingClassifier(estimators = [('l',l),('d',d),('r',r),('g',g),('k',k),('nb',nb)],
                      voting = 'hard', n_jobs = 5)

In [24]:
def classifier(ye):
    print(ye)
    clf = ye.fit(xtrain,ytrain)
    clf.predict(droplabel)
    print('Accuracy of train: ', ye.score(xtrain,ytrain))
    print('Accuracy of test: ', ye.score(xtest,ytest))
    score = cvs(ye,xtest,ytest,cv=5, scoring='accuracy')
    print('cross val score: ', score.mean())

In [25]:
classifier(LogisticRegression())

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
Accuracy of train:  0.898876404494382
Accuracy of test:  0.8791946308724832
cross val score:  0.839006303299963


In [26]:
classifier(DecisionTreeClassifier())

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
Accuracy of train:  1.0
Accuracy of test:  0.8791946308724832
cross val score:  0.8594660734149056


In [27]:
classifier(RandomForestClassifier())

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
Accuracy of train:  0.9955056179775281
Accuracy of test:  0.8926174496644296
cross val score:  0.8656878012606601


In [28]:
classifier(GradientBoostingClassifier())

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)
Accuracy of train:  0.9685393258426966
Accuracy of test:  0.912751677852349
cross val score:  0.8719243604004451


In [29]:
classifier(KNeighborsClassifier())

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')
Accuracy of train:  0.9483146067415731
Accuracy of test:  0.9194630872483222
cross val score:  0.8454430849091583


In [30]:
classifier(MultinomialNB())

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
Accuracy of train:  0.9775280898876404
Accuracy of test:  0.9060402684563759
cross val score:  0.8456729699666296


In [31]:
classifier(vc)

VotingClassifier(estimators=[('l', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)), ('d', DecisionT...
           weights='uniform')), ('nb', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         flatten_transform=None, n_jobs=5, voting='hard', weights=None)


/Users/jedvillareal/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/jedvillareal/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/jedvillareal/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Accuracy of train:  0.9910112359550561
Accuracy of test:  0.9060402684563759


/Users/jedvillareal/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/jedvillareal/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/jedvillareal/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/jedvillareal/anaconda3/envs/ipykernel_py3/lib/python3.6/site

cross val score:  0.8719095291064145


/Users/jedvillareal/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


#### Extracting Ypred

In [38]:
knn = vc.fit(xtrain,ytrain)
ypred = knn.predict(droplabel)
ypred

/Users/jedvillareal/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1.])

In [91]:
len(ypred)

25

In [92]:
len(toadd)

25

In [93]:
results = pd.DataFrame(columns=['text','label'])
results['text'] = toadd
results['label'] = ypred
results.to_csv('kanye.csv')